In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report

# Carregar os arquivos
train_df = pd.read_csv('dataset_nao_tratado/train.csv')
test_df = pd.read_csv('dataset_nao_tratado/test.csv')

# Definir o limite de 50% para exclusão de colunas
threshold = 0.5

# Remover colunas com mais de 50% de valores ausentes em ambos os datasets
train_df = train_df.loc[:, train_df.isnull().mean() <= threshold]

# Alinhar as colunas entre treino e teste (manter apenas colunas comuns)
common_columns = train_df.columns.intersection(test_df.columns)
train_df = train_df[common_columns]
test_df = test_df[common_columns]

# Calcular a mediana das colunas do conjunto de treino
train_medians = train_df.median()

# Preencher os valores ausentes no conjunto de treino usando a mediana calculada
train_df.fillna(train_medians, inplace=True)

# Preencher os valores ausentes no conjunto de teste usando a mediana do conjunto de treino
test_df.fillna(train_medians, inplace=True)

# Função para substituir outliers pela mediana
def replace_outliers_with_median(df, lower_quantile=0.25, upper_quantile=0.75, factor=1.5):
    for col in df.select_dtypes(include=np.number).columns:
        Q1 = df[col].quantile(lower_quantile)
        Q3 = df[col].quantile(upper_quantile)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        median = df[col].median()
        
        # Substituir os valores que estão fora dos limites pelo valor da mediana
        df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), median, df[col])
    return df

# Aplicar a função de substituição dos outliers nos dados de treino e teste
X_train = replace_outliers_with_median(train_df.drop('y', axis=1))
y_train = train_df['y']
X_test = replace_outliers_with_median(test_df.drop('y', axis=1))
y_test = test_df['y']

# Escalonar os dados usando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converter os arrays escalonados de volta para DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
# Definir o modelo KNN com os melhores parâmetros
best_knn = KNeighborsClassifier(n_neighbors=40, weights='distance', metric='manhattan')

# Treinar o modelo com o conjunto de treino filtrado
best_knn.fit(X_train_scaled, y_train)

# Avaliar o modelo no conjunto de treino (autoavaliação para verificar o ajuste no treino)
y_train_pred = best_knn.predict(X_train_scaled)
print("Acurácia no conjunto de treino:", best_knn.score(X_train_scaled, y_train))
print("Relatório de classificação (conjunto de treino):")
print(classification_report(y_train, y_train_pred))

# Avaliar o desempenho no conjunto de teste com features selecionadas
y_test_pred = best_knn.predict(X_test_scaled)
print("\nAcurácia no conjunto de teste:", best_knn.score(X_test_scaled, y_test))
print("Relatório de classificação (conjunto de teste):")
print(classification_report(y_test, y_test_pred))

# Matriz de Confusão para o conjunto de teste com features selecionadas
conf_matrix = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste com Features Selecionadas')
plt.show()

# Calcular o ROC AUC Score para o conjunto de teste com features selecionadas
roc_auc = roc_auc_score(y_test, best_knn.predict_proba(X_test_scaled)[:, 1])
print("ROC AUC Score no conjunto de teste com features selecionadas:", roc_auc)


In [ ]:
# Definir o modelo KNN com os melhores parâmetros
best_knn = KNeighborsClassifier(n_neighbors=100, weights='distance', metric='manhattan')

# Treinar o modelo com o conjunto de treino filtrado
best_knn.fit(X_train_scaled, y_train)

# Avaliar o modelo no conjunto de treino (autoavaliação para verificar o ajuste no treino)
y_train_pred = best_knn.predict(X_train_scaled)
print("Acurácia no conjunto de treino:", best_knn.score(X_train_scaled, y_train))
print("Relatório de classificação (conjunto de treino):")
print(classification_report(y_train, y_train_pred))

# Avaliar o desempenho no conjunto de teste com features selecionadas
y_test_pred = best_knn.predict(X_test_scaled)
print("\nAcurácia no conjunto de teste:", best_knn.score(X_test_scaled, y_test))
print("Relatório de classificação (conjunto de teste):")
print(classification_report(y_test, y_test_pred))

# Matriz de Confusão para o conjunto de teste com features selecionadas
conf_matrix = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste com Features Selecionadas')
plt.show()

# Calcular o ROC AUC Score para o conjunto de teste com features selecionadas
roc_auc = roc_auc_score(y_test, best_knn.predict_proba(X_test_scaled)[:, 1])
print("ROC AUC Score no conjunto de teste com features selecionadas:", roc_auc)

In [ ]:
# Definir o modelo KNN com os melhores parâmetros
best_knn = KNeighborsClassifier(n_neighbors=50, weights='uniform', metric='manhattan')

# Treinar o modelo com o conjunto de treino filtrado
best_knn.fit(X_train_scaled, y_train)

# Avaliar o modelo no conjunto de treino (autoavaliação para verificar o ajuste no treino)
y_train_pred = best_knn.predict(X_train_scaled)
print("Acurácia no conjunto de treino:", best_knn.score(X_train_scaled, y_train))
print("Relatório de classificação (conjunto de treino):")
print(classification_report(y_train, y_train_pred))

# Avaliar o desempenho no conjunto de teste com features selecionadas
y_test_pred = best_knn.predict(X_test_scaled)
print("\nAcurácia no conjunto de teste:", best_knn.score(X_test_scaled, y_test))
print("Relatório de classificação (conjunto de teste):")
print(classification_report(y_test, y_test_pred))

# Matriz de Confusão para o conjunto de teste com features selecionadas
conf_matrix = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste com Features Selecionadas')
plt.show()

# Calcular o ROC AUC Score para o conjunto de teste com features selecionadas
roc_auc = roc_auc_score(y_test, best_knn.predict_proba(X_test_scaled)[:, 1])
print("ROC AUC Score no conjunto de teste com features selecionadas:", roc_auc)

In [ ]:
# Definir o modelo KNN com os melhores parâmetros
best_knn = KNeighborsClassifier(n_neighbors=100, weights='distance', metric='manhattan')

# Treinar o modelo com o conjunto de treino
best_knn.fit(X_train_scaled, y_train)

# Definir um threshold para a classe positiva
threshold = 0.6

# Prever as probabilidades no conjunto de teste
y_test_proba = best_knn.predict_proba(X_test_scaled)[:, 1]  # Probabilidades para a classe positiva

# Aplicar o threshold
y_test_pred_threshold = np.where(y_test_proba >= threshold, 1, 0)

# Avaliar o desempenho com o novo threshold
print("\nAcurácia no conjunto de teste com threshold:", np.mean(y_test_pred_threshold == y_test))
print("Relatório de classificação (conjunto de teste) com threshold:")
print(classification_report(y_test, y_test_pred_threshold))

# Matriz de Confusão para o conjunto de teste com threshold aplicado
conf_matrix = confusion_matrix(y_test, y_test_pred_threshold)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste com Threshold Aplicado')
plt.show()

# Calcular o ROC AUC Score para o conjunto de teste com threshold
roc_auc = roc_auc_score(y_test, y_test_proba)
print("ROC AUC Score no conjunto de teste com threshold aplicado:", roc_auc)


In [ ]:
# Definir o modelo KNN com os melhores parâmetros
best_knn = KNeighborsClassifier(n_neighbors=70, weights='distance', metric='manhattan')

# Treinar o modelo com o conjunto de treino
best_knn.fit(X_train_scaled, y_train)

# Definir um threshold para a classe positiva
threshold = 0.6

# Prever as probabilidades no conjunto de teste
y_test_proba = best_knn.predict_proba(X_test_scaled)[:, 1]  # Probabilidades para a classe positiva

# Aplicar o threshold
y_test_pred_threshold = np.where(y_test_proba >= threshold, 1, 0)

# Avaliar o desempenho com o novo threshold
print("\nAcurácia no conjunto de teste com threshold:", np.mean(y_test_pred_threshold == y_test))
print("Relatório de classificação (conjunto de teste) com threshold:")
print(classification_report(y_test, y_test_pred_threshold))

# Matriz de Confusão para o conjunto de teste com threshold aplicado
conf_matrix = confusion_matrix(y_test, y_test_pred_threshold)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste com Threshold Aplicado')
plt.show()

# Calcular o ROC AUC Score para o conjunto de teste com threshold
roc_auc = roc_auc_score(y_test, y_test_proba)
print("ROC AUC Score no conjunto de teste com threshold aplicado:", roc_auc)

In [ ]:
# Definir o modelo KNN com os melhores parâmetros
best_knn = KNeighborsClassifier(n_neighbors=1024, weights='distance', metric='euclidean', algorithm='brute', p=2)

# Treinar o modelo com o conjunto de treino
best_knn.fit(X_train_scaled, y_train)

# Definir um threshold para a classe positiva
threshold = 0.8

# Prever as probabilidades no conjunto de teste
y_test_proba = best_knn.predict_proba(X_test_scaled)[:, 1]  # Probabilidades para a classe positiva

# Aplicar o threshold
y_test_pred_threshold = np.where(y_test_proba >= threshold, 1, 0)

# Avaliar o desempenho com o novo threshold
print("\nAcurácia no conjunto de teste com threshold:", np.mean(y_test_pred_threshold == y_test))
print("Relatório de classificação (conjunto de teste) com threshold:")
print(classification_report(y_test, y_test_pred_threshold))

# Matriz de Confusão para o conjunto de teste com threshold aplicado
conf_matrix = confusion_matrix(y_test, y_test_pred_threshold)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste com Threshold Aplicado')
plt.show()

# Calcular o ROC AUC Score para o conjunto de teste com threshold
roc_auc = roc_auc_score(y_test, y_test_proba)
print("ROC AUC Score no conjunto de teste com threshold aplicado:", roc_auc)

In [ ]:
# Definir o modelo KNN com os melhores parâmetros
best_knn = KNeighborsClassifier(n_neighbors=1024, weights='distance', metric='euclidean', algorithm='brute', p=2)

# Treinar o modelo com o conjunto de treino
best_knn.fit(X_train_scaled, y_train)

# Definir um threshold para a classe positiva
threshold = 0.8

# Prever as probabilidades no conjunto de teste
y_test_proba = best_knn.predict_proba(X_test_scaled)[:, 1]  # Probabilidades para a classe positiva

# Aplicar o threshold
y_test_pred_threshold = np.where(y_test_proba >= threshold, 1, 0)

# Avaliar o desempenho com o novo threshold
print("\nAcurácia no conjunto de teste com threshold:", np.mean(y_test_pred_threshold == y_test))
print("Relatório de classificação (conjunto de teste) com threshold:")
print(classification_report(y_test, y_test_pred_threshold))

# Matriz de Confusão para o conjunto de teste com threshold aplicado
conf_matrix = confusion_matrix(y_test, y_test_pred_threshold)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Teste com Threshold Aplicado')
plt.show()

# Calcular o ROC AUC Score para o conjunto de teste com threshold
roc_auc = roc_auc_score(y_test, y_test_proba)
print("ROC AUC Score no conjunto de teste com threshold aplicado:", roc_auc)